In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mibitrans.data.parameters import AttenuationParameters
from mibitrans.data.parameters import HydrologicalParameters
from mibitrans.data.parameters import ModelParameters
from mibitrans.data.parameters import SourceParameters
from mibitrans.transport.models import Anatrans
from mibitrans.transport.models import Mibitrans
from mibitrans.visualize.animation import animate_1d

### Exact solution versus untruncated solution

\begin{equation}
C(x,y,z,t) = \sum_{i=1}^{n}\left(C^*_{0,i}\frac{x}{8\sqrt{\pi D^{'}_{x}}}\exp(-\gamma t)
\cdot \int_{0}^{t}\left[\frac{1}{\tau^{\frac{3}{2}}} \exp\left((\gamma - \lambda_{EFF})\tau - \frac{(x-v^{'}\tau)^2}{4D^{'}_{x}\tau}\right)
\cdot \left\{ERFC\left(\frac{y-Y_i}{2 \sqrt{D^{'}_{y}\tau}}\right)-ERFC\left(\frac{y+Y_i}{2 \sqrt{D^{'}_{y}\tau}}\right) \right\}
\cdot \left\{ERFC\left(\frac{z-Z}{2 \sqrt{D^{'}_{z}\tau}}\right)-ERFC\left(\frac{z+Z}{2 \sqrt{D^{'}_{z}\tau}}\right) \right\}\right] d\tau \right)
\end{equation}

In [ ]:
hydro = HydrologicalParameters(
    velocity=0.277, # Flow velocity [m/d]
    porosity=0.25,           # Effective soil porosity [-]
    alpha_x=10,         # Longitudinal dispersivity, in [m]
    alpha_y=1,          # Transverse horizontal dispersivity, in [m]
    alpha_z=0.1                # Transverse vertical dispersivity, in [m]
)

att = AttenuationParameters(
    retardation=1,
    # Molecular diffusion, in [m2/day]
    diffusion=0,
    # Contaminant half life, in [days]
    half_life=0#5*365
)

source = SourceParameters(
    source_zone_boundary=np.array([10]),
    source_zone_concentration=np.array([11]),
    depth=2.5,
    total_mass="inf"
)

model = ModelParameters(
    # Model extent in the longitudinal (x) direction in [m].
    model_length = 800,
    # Model extent in the transverse horizontal (y) direction in [m].
    model_width = 150,
    # Model duration in [days].
    model_time = 5*365,
    # Model grid discretization step size in the longitudinal (x) direction, in [m].
    dx = 2,
    # Model grid discretization step size in the transverse horizontal (y) direction, in [m].
    dy = 1,
    # Model time discretization step size, in [days]
    dt = 25
)

In [ ]:
mbt_object = Mibitrans(hydro, att, source, model)
mbt_object.run()

ana_object = Anatrans(hydro, att, source, model)
ana_object.run()

In [ ]:
%matplotlib notebook

In [ ]:
ani = animate_1d(x_axis_parameter=mbt_object.y,
                 y_axis_parameter=[mbt_object.cxyt[:,:,25], mbt_object.cxyt[:,:,50],
                                   mbt_object.cxyt[:,:,100],mbt_object.cxyt[:,:,200],
                                   ana_object.cxyt[:,:,25], ana_object.cxyt[:,:,50],
                                   ana_object.cxyt[:,:,100], ana_object.cxyt[:,:,200]],
                 time_parameter=mbt_object.t,
                 y_colors=["darkgreen", "limegreen", "cornflowerblue", "blue",
                           "green", "greenyellow", "darkturquoise", "dodgerblue"],
                 y_names=["Mibitrans x=50m", "Mibitrans x=100m", "Mibitrans x=200m", "Mibitrans x=400m",
                          "Anatrans x=50m", "Anatrans x=100m", "Anatrans x=200m", "Anatrans x=400m"],
                 linestyle=["-", "-", "-", "-", "--", "--", "--", "--"])
plt.xlabel("y-position [m]")
plt.show()


In [ ]:
ani1 = animate_1d(x_axis_parameter=mbt_object.x,
                  y_axis_parameter=[mbt_object.cxyt[:,75,:], mbt_object.cxyt[:,84,:],
                                    mbt_object.cxyt[:,100,:], mbt_object.cxyt[:,125,:],
                                   ana_object.cxyt[:,75,:], ana_object.cxyt[:,84,:],
                                    ana_object.cxyt[:,100,:], ana_object.cxyt[:,125,:]],
                  time_parameter=mbt_object.t,
                  y_colors=["darkgreen", "limegreen", "cornflowerblue", "blue",
                            "green", "greenyellow", "darkturquoise", "dodgerblue"],
                  y_names=["Mibitrans y=0m", "Mibitrans y=9m", "Mibitrans y=25m", "Mibitrans y=50m",
                           "Anatrans y=0m", "Anatrans y=9m", "Anatrans y=25m", "Anatrans y=50m"],
                  linestyle=["-", "-", "-", "-", "--", "--", "--", "--"])
plt.xlabel("x-position [m]")
plt.show()


In [ ]:
plt.figure()
mbt_object.breakthrough(50, 0, color="darkgreen", linestyle="-", label="Mibitrans 50m")
mbt_object.breakthrough(100, 0, color="limegreen", linestyle="-", label="Mibitrans 100m")
mbt_object.breakthrough(200, 0, color="cornflowerblue", linestyle="-", label="Mibitrans 200m")
mbt_object.breakthrough(400, 0, color="blue", linestyle="-", label="Mibitrans 400m")
ana_object.breakthrough(50, 0, color="green", linestyle="--", label="Anatrans 50m")
ana_object.breakthrough(100, 0, color="greenyellow", linestyle="--", label="Anatrans 100m")
ana_object.breakthrough(200, 0, color="darkturquoise", linestyle="--", label="Anatrans 200m")
ana_object.breakthrough(400, 0, color="dodgerblue", linestyle="--", label="Anatrans 400m")
plt.title("Breakthrough plot of Mibitrans and Anatrans model, at various x locations.")
plt.legend()
plt.show()

#### Comparing model differences for various flow velocities and dispersivities

In [ ]:
parameter_list = [0.1, 0.5, 1, 5, 10]
output_list_mbt = []
output_list_ana = []
for par in parameter_list:
    print(f"starting run with par = {par}")
    mbt_object = Mibitrans(hydro, att, source, model)
    mbt_object.hydrological_parameters.alpha_x = par
    mbt_object.hydrological_parameters.alpha_y = par / 10
    mbt_object.hydrological_parameters.alpha_z = par / 100
    mbt_object.run()
    output_list_mbt.append(mbt_object.cxyt)

    ana_object = Anatrans(hydro, att, source, model)
    ana_object.hydrological_parameters.alpha_x = par
    ana_object.hydrological_parameters.alpha_y = par / 10
    ana_object.hydrological_parameters.alpha_z = par / 100
    ana_object.run()
    output_list_ana.append(ana_object.cxyt)

In [ ]:
colors_mbt = ["darkgreen", "limegreen", "cornflowerblue", "blue", "indigo"]
colors_ana = ["green", "greenyellow", "darkturquoise", "dodgerblue", "darkviolet"]
plt.figure()
for i, conc in enumerate(output_list_mbt):
    plt.plot(mbt_object.x, conc[-1,75,:], color=colors_mbt[i], label=f"Mibitrans a={parameter_list[i]}m")
for i, conc in enumerate(output_list_ana):
    plt.plot(ana_object.x, conc[-1,75,:], color=colors_ana[i], linestyle="--", label=f"Anatrans a={parameter_list[i]}m")
plt.legend()
plt.show()

In [ ]:
colors_mbt = ["darkgreen", "limegreen", "cornflowerblue", "blue", "indigo"]
colors_ana = ["green", "greenyellow", "darkturquoise", "dodgerblue", "darkviolet"]
plt.figure()
for i, conc in enumerate(output_list_mbt):
    plt.plot(mbt_object.y, conc[-1,:,175], color=colors_mbt[i],
             label=f"Mibitrans a={parameter_list[i]}m")
for i, conc in enumerate(output_list_ana):
    plt.plot(ana_object.y, conc[-1,:,175], color=colors_ana[i],
             linestyle="--", label=f"Anatrans a={parameter_list[i]}m")
plt.legend()
plt.show()

In [ ]:
parameter_list_v = [0.1, 0.1, 0.1, 0.1, 0.1]
parameter_list_a = [10, 5, 2, 1, 0.5]
output_list_mbt_v = []
output_list_ana_v = []
for i in range(len(parameter_list_v)):
    parv = parameter_list_v[i]
    para = parameter_list_a[i]
    print(f"starting run with parv = {parv} and para = {para}")
    mbt_object = Mibitrans(hydro, att, source, model)
    mbt_object.hydrological_parameters.velocity = parv
    mbt_object.hydrological_parameters.alpha_x = para
    mbt_object.hydrological_parameters.alpha_y = para / 10
    mbt_object.hydrological_parameters.alpha_z = 0#para / 100
    mbt_object.run()
    output_list_mbt_v.append(mbt_object.cxyt)

    ana_object = Anatrans(hydro, att, source, model)
    ana_object.hydrological_parameters.velocity = parv
    ana_object.hydrological_parameters.alpha_x = para
    ana_object.hydrological_parameters.alpha_y = para / 10
    ana_object.hydrological_parameters.alpha_z = 0#para / 100
    ana_object.run()
    output_list_ana_v.append(ana_object.cxyt)

In [ ]:
colors_mbt = ["darkgreen", "limegreen", "cornflowerblue", "blue", "indigo"]
colors_ana = ["green", "greenyellow", "darkturquoise", "dodgerblue", "darkviolet"]
plt.figure()
for i, conc in enumerate(output_list_mbt_v):
    plt.plot(mbt_object.x, conc[-1,75,:], color=colors_mbt[i],
             label=f"Mbt v={parameter_list_v[i]}m/d, a={parameter_list_a[i]}m")
for i, conc in enumerate(output_list_ana_v):
    plt.plot(ana_object.x, conc[-1,75,:], color=colors_ana[i],
             linestyle="--", label=f"Ana v={parameter_list_v[i]}m/d, a={parameter_list_a[i]}m")
plt.xlim((0,300))
plt.legend()
plt.show()

In [ ]:
colors_mbt = ["darkgreen", "limegreen", "cornflowerblue", "blue", "indigo"]
colors_ana = ["green", "greenyellow", "darkturquoise", "dodgerblue", "darkviolet"]
plt.figure()
for i, conc in enumerate(output_list_mbt_v):
    plt.plot(mbt_object.y, conc[-1, :, 75], color=colors_mbt[i], label=f"Mibitrans a={parameter_list_v[i]}m")
for i, conc in enumerate(output_list_ana_v):
    plt.plot(ana_object.y, conc[-1, :, 75], color=colors_ana[i], linestyle="--",
             label=f"Anatrans a={parameter_list[i]}m")
plt.legend()
plt.show()

In [ ]:
mbtv=output_list_mbt_v
anav=output_list_ana_v
ani = animate_1d(x_axis_parameter=mbt_object.x,
                 y_axis_parameter=[mbtv[0][:,75,:], mbtv[1][:,75,:],mbtv[2][:,75,:],
                                   mbtv[3][:,75,:], mbtv[4][:,75,:],
                                   anav[0][:,75,:], anav[1][:,75,:], anav[2][:,75,:],
                                   anav[3][:,75,:], anav[4][:,75,:]],
                 time_parameter=mbt_object.t,
                 y_colors=["darkgreen", "limegreen", "cornflowerblue", "blue", "indigo",
                           "green", "greenyellow", "darkturquoise", "dodgerblue", "violet"],
                 y_names=["Mibitrans a=0.1", "Mibitrans a=0.5", "Mibitrans a=1", "Mibitrans a=5", "Mibitrans a=10",
                          "Anatrans a=0.1", "Anatrans a=0.5", "Anatrans a=1", "Anatrans a=5", "Anatrans a=10"],
                 linestyle=["-", "-", "-", "-", "-", "--", "--", "--", "--", "--"])
plt.xlabel("y-position [m]")
plt.xlim((0,350))
plt.show()


In [ ]:
mbtv=output_list_mbt_v
anav=output_list_ana_v
ani = animate_1d(x_axis_parameter=mbt_object.y,
                 y_axis_parameter=[mbtv[0][:,:,50], mbtv[1][:,:,50], mbtv[2][:,:,50],
                                   mbtv[3][:,:,50], mbtv[4][:,:,50],
                                   anav[0][:,:,50], anav[1][:,:,50], anav[2][:,:,50],
                                   anav[3][:,:,50], anav[4][:,:,50]],
                 time_parameter=mbt_object.t,
                 y_colors=["darkgreen", "limegreen", "cornflowerblue", "blue", "indigo",
                           "green", "greenyellow", "darkturquoise", "dodgerblue", "violet"],
                 y_names=["Mibitrans a=0.1", "Mibitrans a=0.5", "Mibitrans a=1", "Mibitrans a=5", "Mibitrans a=10",
                          "Anatrans a=0.1", "Anatrans a=0.5", "Anatrans a=1", "Anatrans a=5", "Anatrans a=10"],
                 linestyle=["-", "-", "-", "-", "-", "--", "--", "--", "--", "--"])
plt.xlabel("y-position [m]")

plt.show()